In [6]:
import numpy as np
import tensorflow as tf
import keras
from keras.layers import Dense,Conv2D,Conv2DTranspose,Input,Reshape,Activation,Lambda
from keras.layers.advanced_activations import LeakyReLU
from keras.optimizers import Adam
from keras.layers import BatchNormalization,Dropout,Flatten
from keras.datasets import mnist
from keras.models import Sequential,Model
import keras.backend as K
import matplotlib.pyplot as plt

class Dataset:
  '''
  Class pour gérér les données
  '''

  def __init__(self,num_labeled,num_classes):
    self.num_labeled = num_labeled
    self.num_classes = num_classes
    (self.x_train,self.y_train),(self.x_test,self.y_test) = mnist.load_data()      #loading mnist data
  
    def normalize_image(x):
      '''
      normaliser une image
      '''
      x = (x.astype(np.float32)-127.5)/127.5
      x = np.expand_dims(x,axis=-1)
      return x
  
    def reshape_labels(y):
      return y.reshape((-1,1))

    self.x_train = normalize_image(self.x_train)
    self.y_train = reshape_labels(self.y_train)
    self.x_test = normalize_image(self.x_test)
    self.y_test = reshape_labels(self.y_test)
 
  def sample_labeled(self,batch_size):
    '''
    générer des données annotées
    '''
    sample_per_class = int(self.num_labeled/self.num_classes)
    imgs = list()
    labels = list()

    for i in range(self.num_classes):
      idx = self.y_train==i                             
      idx = [i for i, x in enumerate(idx) if x]
      x_classes= self.x_train[idx]
      y_classes = self.y_train[idx]

      index = np.arange(sample_per_class)
      x_imgs = x_classes[index]                
      y_imgs = y_classes[index]
      
      [imgs.append(x_imgs[j]) for j in index ]
      [labels.append(y_imgs[j]) for j in index]
 
    imgs = np.asarray(imgs) 
    labels = np.asarray(labels)
 
    return imgs,labels
 
  def sample_unlabeled(self,batch_size):
    '''
    générer des images non étiquetées
    '''
    idx = np.random.randint(self.num_labeled,self.x_train.shape[0],batch_size)
    imgs = self.x_train[idx]
    return imgs
 
  def get_training(self):
      '''
      récuperer les données d'apprentissage
      '''
      x_train = self.x_train[range(self.num_labeled)]  
      y_train = self.y_train[range(self.num_labeled)]
      return x_train ,y_train

  def get_testing(self):
    '''
    récuperer les données de test
    '''
    return self.x_test,self.y_test
 


In [2]:
from keras.utils import to_categorical
class GAN:
  '''
  Class pour gérer les modèles GAN
  '''

  def __init__(self,z_dim = 100,num_classes = 10):
    '''
    Créer tous les modèles
    '''
    img_rows = 28
    img_cols = 28
    channels = 1
    
    self.img_shape = (img_rows,img_cols,channels)

    self.z_dim=z_dim
    self.num_classes=num_classes
    

    self.discriminator = self.build_discriminator(self.img_shape)

    self.discriminator_supervised = self.build_discriminator_supervised(self.discriminator)
    self.discriminator_supervised.compile(optimizer= Adam(lr=0.0002, beta_1=0.5),loss="categorical_crossentropy",metrics=['accuracy'])

    self.discriminator_unsupervised = self.build_discriminator_unsupervised(self.discriminator)
    self.discriminator_unsupervised.compile(optimizer = Adam(lr=0.0002, beta_1=0.5),loss='binary_crossentropy',metrics=['accuracy'])

    self.generator = self.build_generator(z_dim)
    
    self.gan = self.build_gan(self.generator,self.discriminator_unsupervised)
    self.gan.compile(optimizer=Adam(learning_rate=0.0002),loss='binary_crossentropy',metrics=['accuracy'])

  def build_generator(self,z_dim):
    '''
    Créer le générateur
    '''
    model = Sequential()

    model.add(Dense(128*7*7,input_dim=z_dim))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Reshape((7,7,128)))

    model.add(Conv2DTranspose(128,kernel_size=4,strides=2,padding='same'))
    model.add(LeakyReLU(alpha=0.2))

    model.add(Conv2DTranspose(128,kernel_size=4,strides=2,padding='same'))
    model.add(LeakyReLU(alpha=0.2))

    model.add(Conv2D(1, (7,7), activation='tanh', padding='same'))

    return model

  def build_discriminator(self,img_shape):
    '''
    Créer le discriminateur
    '''

    inp = Input(shape=img_shape)

    X = Conv2D(128,kernel_size=(3,3),strides=2,input_shape=img_shape,padding='same')(inp)
    X = LeakyReLU(alpha=0.2)(X)

    X = Conv2D(128,kernel_size=(3,3),strides=2,padding='same')(X)
    X = LeakyReLU(alpha=0.2)(X)

    X = Conv2D(128,kernel_size=(3,3),strides=2,padding='same')(X)
    X = LeakyReLU(alpha=0.2)(X)

    X = Flatten(name="flatten")(X)
    X = Dropout(0.4)(X)
    X = Dense(num_classes)(X)
    model = Model(inputs=inp,outputs=X)

    return model

  def build_discriminator_unsupervised(self,discriminator):
    '''
    créer la partie non supervisée du discriminateur
    '''
    model = Sequential()
    model.add(discriminator)
    def custom_activation(x):
          
      prediction = 1.0 - (1.0 /
                            (K.sum(K.exp(x), axis=-1, keepdims=True) + 1.0))
      return prediction

    model.add(Lambda(custom_activation))
    
    return model

  def build_discriminator_supervised(self,discriminator):
    '''
    créer la partie supervisée du discriminateur
    '''
    model = Sequential()
    model.add(discriminator)
    model.add(Activation('softmax'))
    return model

  def build_gan(self,generator,discriminator):
    '''
    Concatener les deux modèles pour avoir un GAN.
    '''
    model = Sequential()
    model.add(generator)
    model.add(discriminator)
    return model

  def train(self,dataset,iterations= 12000,batch_size= 100,sample_interval= 500):
    '''
    Entraîner le modèle gan complet
    '''

    z_dim=self.z_dim

    supervised_losses = []
    iteration_checkpoints = []
    accuracies = []
    val_losses = []
    val_accuracies=[]

    half = int(batch_size/2)
    real = np.ones((half,1))
    full_real=np.ones((batch_size,1))
    fake = np.zeros((half,1))

    x_test,y_test=dataset.get_testing()
    y_test=to_categorical(y_test,num_classes=num_classes)

    


    for iteration in range(iterations):
      imgs,labels = dataset.sample_labeled(half)
      labels = to_categorical(labels,num_classes=self.num_classes)
      unlabeled_imgs = dataset.sample_unlabeled(half)

      z = np.random.normal(0,1,(half,z_dim))
      
      fake_imgs = self.generator.predict(z)

      d_supervised_loss,accuracy = self.discriminator_supervised.train_on_batch(imgs,labels)
      d_unsupervised_loss_real = self.discriminator_unsupervised.train_on_batch(unlabeled_imgs,real)
      d_unsupervised_loss_fake = self.discriminator_unsupervised.train_on_batch(fake_imgs,fake)
      d_unsupervised_loss = 0.5*np.add(d_unsupervised_loss_real,d_unsupervised_loss_fake)

      z = np.random.normal(0,1,(batch_size,z_dim))
      fake_imgs = self.generator.predict(z)
      generator_loss = self.gan.train_on_batch(z,full_real)

      
      if (iteration+1) % sample_interval ==0:
        supervised_losses.append(d_supervised_loss)
        accuracies.append(100*accuracy)
        iteration_checkpoints.append(iteration+1)
        val_loss = self.discriminator_supervised.evaluate(x=x_test,y=y_test,verbose=0)
        val_losses.append(val_loss[0])
        val_accuracies.append(val_loss[1])
        print("Iteration No.:",iteration+1,end=",")
        print("Discriminator Supervised Loss:",d_supervised_loss,end=',')
        print('Generator Loss:',generator_loss,end=",")
        print('Discriminator Unsuperived Loss:',d_unsupervised_loss,sep=',')
        print('val_loss:',val_loss,sep=',')
        print('Accuracy Supervised:',100*accuracy)

        ''' filename1 = 'generator_%04d.h5' % (iteration+1)
        self.generator.save(filename1)
        # save the classifier model
        filename2 = 'classifier_%04d.h5' % (iteration+1)
        self.discriminator_supervised.save(filename2) '''

In [ ]:
'''
Créer le dataset et lancer l'entraînement
'''
num_labeled = 100
num_classes = 10

dataset = Dataset(num_labeled,num_classes)
gan=GAN()

gan.train(dataset=dataset,iterations=200,sample_interval=100)

In [ ]:
'''
Tester le modèle après entraînement
'''

x_test,y_test=dataset.get_testing()
y_test=to_categorical(y_test,num_classes=num_classes)
loss,accuracy=gan.discriminator_supervised.evaluate(x=x_test,y=y_test,verbose=0)
print(loss,accuracy)

In [ ]:
'''
Vérifier la qualité des images par le générateur
'''

n_images,zdim=100,100
z = np.random.normal(0,1,(n_images,zdim))
gen_imgs=gan.generator.predict(z)

M = 10
fig, axs = plt.subplots(M, M, figsize=(20,20))

for i in range(M):
    for j in range(M):
        axs[i,j].imshow(gen_imgs[10*i+j,:,:,0],cmap='gray')
        axs[i,j].axis('off')